# Read gene file

In [1]:
# f = open('ce11.fa', 'r')
# gene_1 = ''
# f.readline()
# for line in f.readlines():
#     gene_1 = gene_1 + line[:-1]
# f.close()

# f = open('cb4.fa', 'r')
# gene_2 = ''
# f.readline()
# for line in f.readlines():
#     gene_2 = gene_2 + line[:-1]
# f.close()

# print(len(gene_1), len(gene_2))

# Generate random gene sequence

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(10)

random_pattern = True
seq_size = 100000

d = ['A', 'C', 'G', 'T']
if random_pattern:
    gene_1, gene_2 = '', ''
    for i in range(seq_size):
        gene_1 += d[random.randint(0, 3)]
    i = 0
    while i < seq_size:
        die = random.random()
        l = random.randint(1,10)
        # delete gene_1 for length l
        if die < 0.06:
            i += l
        # insert gene_1 for length l
        elif die < 0.12:
            for j in range(l):
                gene_2 += d[random.randint(0, 3)]
            i += 1
        # change 1 gene
        elif die < 0.18:
            gene_2 += d[random.randint(0, 3)]
            i += 1
        else:
            gene_2 += gene_1[i]
            i += 1
    while len(gene_2) < seq_size:
        gene_2 += d[random.randint(0, 3)]
            
    print(len(gene_1), len(gene_2))
    # print(gene_1)
    # print(gene_2)

100000 100000


# Build seed-look-up on reference sequence

In [3]:
k = 8
# base = ['A', 'C', 'G', 'T']
# seeds = []
# for i in range(4**k):
#     seed = ''
#     for j in range(k):
#         seed += base[i%4]
#         i = i // 4
#     seeds.append(seed)
# print(seeds)

# seed_to_index = {seeds[i]:i for i in range(len(seeds))}

def seed_to_index(seed):
    base = {'A':0, 'C':1, 'G':2, 'T':3}
    index = 0
    for i in range(len(seed)):
        index += (4**i)*base[seed[i]]
    return index

# print(seeds[100],seed_index(seeds[100]))

In [4]:
# position_table[i] is a list, memory start positions of seeds[i] in reference sequence
position_table = [[] for i in range(4**k)]
for i in range(len(gene_1)-k+1):
    cut = gene_1[i:i+k]
    index = seed_to_index(cut)
    position_table[index].append(i)
print(position_table)

[[45161, 45162, 68659, 74807], [45160, 63871, 74806], [29700, 49776], [6290, 68658], [], [12632, 20929, 63870, 74805], [], [45159], [49775], [], [17102, 29699], [46731, 73486], [1079, 5655, 6289, 42749, 71630], [54027, 68657], [], [19381], [], [16628, 57458], [], [8571], [32830, 64868], [14787, 62270, 63869], [18245, 45730, 60435, 74804], [12631, 18094, 20928], [], [8600, 29411, 46412, 73100], [72055], [24180, 65694], [14883, 45158, 67518], [], [99381], [3383, 57084], [], [49797], [], [49774, 93452, 99394], [], [88612], [], [30715, 65120, 81580], [98334], [17101, 20765, 24691, 29698, 64709, 77609], [60234], [30699], [33274, 46730], [73485], [], [69839], [], [1078, 91827], [5654, 58018], [6288, 42748, 71629], [50303, 68656, 90523], [43580], [54026, 83886], [38218], [54539], [], [34553, 44494, 70343, 81426, 89561], [41108, 90062], [1455, 77436, 84069], [80376], [19380], [], [], [66222, 83282], [5232], [41695, 61276], [44574], [16627], [], [57457], [], [70981], [55097], [7025], [8570], []

# D-Soft filtering

In [5]:
# reference: gene_1, query: gene_2
bin_size = 500
chunk_size = 500
threshold = 20

bin_num = len(gene_1) // bin_size
chunk_num = len(gene_2) // chunk_size

final_hits = []

# iterate every chunks
hit_positions = []
# hits count & last hit position of each bins
hit_num_in_bins = [0 for i in range(bin_num)]
last_hit_pos = [None for i in range(bin_num)]

for i in range(chunk_num):
    chunk = gene_2[i*chunk_size:(i+1)*chunk_size]
    # print(f'chunk_{i}: {chunk}')
    for j in range(len(chunk)-k+1):
        cut = chunk[j:j+k]
        seed_index = seed_to_index(cut)
        hits = np.array(position_table[seed_index])
        # for h in hits:
        #    print(f'hit at {i*chunk_size+j, h} {gene_1[h:h+k] == cut}')
        for hit in hits:
            if (hit-j) > 0:
                hit_num_in_bins[(hit-j)//bin_size] += 1
                last_hit_pos[(hit-j)//bin_size] = (i*chunk_size+j, hit) # (position in gene_2, position in gene_1)
    print(hit_num_in_bins)
    # print(last_hit_pos)
    for i in range(bin_num):
        if hit_num_in_bins[i] >= threshold:
            final_hits.append(last_hit_pos[i])
    hit_num_in_bins = [0 for i in range(bin_num)]
    
print(final_hits)
print(len(final_hits))

[109, 3, 4, 6, 1, 2, 6, 1, 2, 1, 6, 4, 8, 2, 6, 3, 4, 6, 2, 3, 8, 8, 1, 2, 1, 4, 4, 4, 6, 4, 5, 3, 1, 2, 5, 6, 4, 2, 5, 2, 8, 3, 1, 6, 10, 5, 3, 1, 9, 2, 8, 2, 3, 2, 1, 1, 7, 3, 5, 4, 0, 0, 4, 8, 1, 0, 6, 10, 4, 10, 3, 7, 0, 4, 3, 1, 8, 1, 1, 6, 3, 2, 2, 7, 1, 6, 2, 8, 5, 2, 7, 2, 0, 1, 1, 6, 2, 4, 4, 0, 3, 5, 4, 6, 5, 2, 3, 8, 7, 8, 3, 3, 4, 2, 4, 5, 2, 6, 1, 1, 2, 2, 5, 2, 2, 9, 2, 2, 8, 2, 12, 6, 0, 1, 0, 5, 5, 9, 2, 6, 3, 4, 8, 3, 4, 6, 5, 3, 8, 3, 4, 0, 3, 3, 5, 1, 10, 0, 9, 4, 3, 4, 4, 4, 2, 5, 1, 4, 5, 8, 7, 2, 2, 3, 5, 4, 3, 6, 1, 8, 1, 5, 3, 7, 3, 9, 8, 3, 3, 3, 1, 3, 2, 4, 7, 3, 2, 3, 0, 2]
[2, 109, 1, 8, 8, 3, 3, 4, 5, 0, 1, 2, 3, 3, 4, 2, 5, 7, 1, 2, 2, 3, 4, 2, 4, 4, 4, 4, 6, 2, 4, 6, 1, 2, 3, 2, 0, 7, 2, 4, 2, 3, 2, 2, 2, 3, 2, 0, 4, 1, 2, 2, 4, 2, 5, 3, 2, 3, 5, 6, 8, 6, 4, 3, 6, 3, 4, 0, 1, 10, 5, 1, 4, 7, 4, 3, 5, 2, 10, 6, 2, 6, 3, 1, 1, 6, 3, 2, 0, 0, 5, 8, 7, 3, 6, 7, 4, 15, 2, 7, 1, 0, 5, 2, 2, 5, 3, 7, 11, 0, 3, 3, 0, 4, 2, 7, 4, 2, 3, 3, 1, 1, 3, 3, 6, 4, 0, 2, 2

[0, 11, 1, 0, 3, 4, 0, 3, 5, 7, 4, 4, 2, 4, 6, 6, 5, 3, 4, 3, 4, 4, 6, 5, 4, 4, 4, 1, 4, 0, 118, 5, 3, 0, 5, 4, 9, 2, 9, 1, 5, 6, 3, 1, 6, 8, 6, 3, 2, 2, 0, 8, 3, 6, 3, 3, 8, 2, 1, 11, 4, 3, 2, 2, 5, 4, 2, 1, 3, 2, 2, 5, 5, 0, 1, 4, 1, 10, 6, 4, 2, 1, 3, 1, 2, 6, 0, 2, 4, 8, 1, 4, 3, 4, 6, 1, 6, 5, 6, 1, 5, 5, 4, 1, 7, 1, 0, 4, 0, 4, 3, 1, 3, 3, 3, 4, 5, 1, 3, 2, 2, 3, 3, 5, 4, 3, 3, 2, 2, 0, 1, 4, 2, 1, 2, 5, 3, 5, 2, 2, 1, 0, 3, 3, 5, 2, 1, 5, 4, 0, 2, 3, 4, 6, 2, 0, 4, 5, 1, 2, 1, 3, 1, 4, 1, 2, 5, 3, 6, 1, 1, 3, 6, 5, 8, 3, 3, 2, 5, 3, 3, 5, 6, 0, 2, 4, 4, 4, 3, 10, 2, 3, 2, 3, 8, 8, 4, 4, 2, 0]
[11, 3, 4, 8, 1, 3, 4, 3, 6, 7, 6, 12, 5, 2, 2, 1, 6, 6, 2, 3, 5, 5, 4, 1, 2, 5, 5, 2, 2, 3, 3, 127, 2, 2, 2, 5, 5, 7, 4, 4, 4, 5, 2, 5, 6, 4, 1, 6, 6, 5, 0, 7, 10, 2, 11, 4, 8, 3, 3, 1, 4, 3, 0, 3, 5, 2, 2, 4, 8, 3, 3, 3, 2, 1, 4, 2, 2, 1, 2, 3, 2, 1, 1, 2, 7, 6, 6, 4, 5, 8, 2, 1, 4, 5, 2, 7, 4, 3, 1, 6, 7, 2, 3, 3, 1, 3, 4, 1, 1, 2, 5, 4, 4, 0, 4, 4, 3, 9, 7, 3, 2, 1, 0, 8, 5, 0, 2, 2, 9,

[8, 2, 3, 4, 1, 5, 0, 1, 8, 2, 5, 1, 4, 1, 4, 3, 2, 3, 0, 4, 2, 1, 6, 1, 4, 3, 4, 4, 2, 4, 1, 0, 2, 3, 3, 2, 6, 5, 4, 9, 9, 4, 4, 6, 2, 6, 5, 4, 1, 8, 4, 3, 1, 3, 1, 4, 2, 3, 4, 2, 3, 7, 4, 4, 4, 5, 2, 4, 3, 5, 1, 5, 6, 4, 3, 3, 0, 2, 9, 5, 4, 1, 4, 4, 5, 5, 0, 8, 2, 7, 1, 5, 2, 2, 5, 3, 9, 4, 2, 8, 1, 3, 0, 1, 7, 4, 4, 1, 114, 7, 4, 3, 1, 1, 8, 6, 1, 2, 4, 9, 0, 0, 2, 3, 3, 1, 13, 7, 1, 6, 2, 3, 3, 3, 3, 3, 3, 6, 5, 3, 6, 3, 3, 5, 1, 4, 2, 9, 0, 2, 0, 0, 2, 6, 9, 2, 6, 0, 3, 2, 0, 5, 1, 2, 1, 8, 1, 4, 3, 0, 3, 2, 3, 3, 0, 2, 4, 5, 3, 8, 5, 3, 4, 6, 2, 4, 3, 0, 5, 2, 4, 2, 4, 3, 1, 4, 4, 4, 6, 1]
[5, 6, 6, 6, 3, 3, 2, 2, 3, 0, 0, 2, 1, 7, 1, 3, 4, 3, 1, 1, 6, 5, 5, 7, 1, 2, 7, 9, 3, 1, 3, 2, 5, 9, 0, 2, 14, 4, 5, 2, 7, 1, 3, 3, 8, 5, 1, 8, 3, 5, 5, 0, 6, 4, 3, 7, 6, 4, 4, 3, 3, 3, 6, 3, 4, 2, 10, 5, 3, 4, 3, 3, 5, 1, 2, 9, 8, 8, 7, 3, 1, 7, 6, 6, 4, 3, 5, 7, 2, 3, 2, 1, 3, 6, 10, 2, 8, 1, 0, 8, 2, 7, 1, 0, 4, 1, 3, 5, 5, 79, 6, 6, 5, 9, 3, 3, 2, 4, 0, 4, 2, 8, 5, 2, 1, 3, 2, 4, 4, 5, 3

[1, 4, 1, 3, 4, 6, 1, 5, 2, 5, 3, 4, 7, 8, 1, 2, 0, 3, 7, 1, 5, 0, 3, 2, 8, 1, 1, 4, 0, 5, 10, 3, 4, 2, 1, 4, 5, 7, 2, 4, 5, 3, 5, 0, 0, 4, 5, 1, 3, 5, 3, 3, 8, 5, 4, 5, 2, 6, 2, 2, 5, 2, 5, 7, 4, 1, 2, 4, 5, 4, 1, 5, 4, 5, 3, 2, 8, 5, 7, 7, 1, 6, 2, 3, 5, 6, 2, 1, 6, 2, 3, 8, 8, 1, 7, 5, 7, 7, 8, 1, 2, 2, 1, 5, 8, 2, 3, 6, 6, 5, 2, 9, 5, 6, 3, 0, 5, 1, 4, 7, 6, 4, 2, 2, 4, 5, 7, 1, 5, 5, 9, 10, 0, 1, 8, 4, 5, 1, 4, 2, 6, 4, 2, 3, 3, 3, 3, 5, 2, 5, 2, 7, 1, 2, 3, 3, 4, 3, 7, 3, 5, 3, 4, 4, 1, 2, 7, 6, 2, 0, 2, 10, 5, 2, 4, 8, 2, 2, 4, 2, 4, 2, 4, 5, 4, 11, 4, 3, 4, 7, 3, 87, 3, 1, 7, 2, 0, 9, 2, 2]
[2, 10, 5, 3, 13, 9, 4, 8, 1, 5, 2, 7, 2, 6, 5, 3, 1, 4, 3, 7, 2, 8, 4, 2, 3, 4, 5, 5, 0, 0, 6, 1, 1, 4, 4, 1, 3, 9, 0, 3, 8, 4, 4, 2, 2, 4, 5, 4, 3, 2, 0, 2, 3, 3, 4, 2, 2, 0, 2, 2, 1, 1, 4, 2, 3, 3, 6, 9, 8, 7, 4, 1, 10, 1, 2, 0, 6, 13, 6, 2, 10, 1, 5, 6, 2, 1, 2, 9, 4, 3, 3, 1, 2, 7, 2, 5, 6, 1, 3, 1, 8, 6, 5, 7, 4, 3, 6, 1, 5, 3, 4, 7, 10, 7, 0, 2, 1, 3, 2, 1, 3, 1, 3, 1, 1, 4, 2, 11, 1,

In [6]:
for i in final_hits:
    print(gene_2[i[0]:i[0]+k], gene_1[i[1]:i[1]+k], gene_2[i[0]:i[0]+k]==gene_1[i[1]:i[1]+k])

AGTATATT AGTATATT True
GTAGAGGA GTAGAGGA True
TGGCATCC TGGCATCC True
TCTGACGC TCTGACGC True
CATTCGGA CATTCGGA True
GCGGCCGT GCGGCCGT True
GAGGGGAT GAGGGGAT True
CCGAGAGT CCGAGAGT True
CGAACATA CGAACATA True
AGTTGTGA AGTTGTGA True
CCGGAAGG CCGGAAGG True
GGCAATGA GGCAATGA True
ACCGTAGA ACCGTAGA True
CACCCAGA CACCCAGA True
GCATATCA GCATATCA True
ACAAGACA ACAAGACA True
ACGTTAGG ACGTTAGG True
TAGAAGCC TAGAAGCC True
TCGTAATA TCGTAATA True
ATCGTTAT ATCGTTAT True
GTCATCAC GTCATCAC True
GCGACCGT GCGACCGT True
CCGTTTTC CCGTTTTC True
GATAAGTA GATAAGTA True
AGCCGAAC AGCCGAAC True
GCCTTTAA GCCTTTAA True
ACGTATTT ACGTATTT True
GTGAAGGA GTGAAGGA True
GGTACTTA GGTACTTA True
CCTGATTT CCTGATTT True
CATCCCCT CATCCCCT True
AGGAATGC AGGAATGC True
TCTAGACT TCTAGACT True
GAGCTGCG GAGCTGCG True
GGTCGCGA GGTCGCGA True
GGGCATCA GGGCATCA True
GCATTATC GCATTATC True
CATGATAA CATGATAA True
GCAATGCC GCAATGCC True
GAAAGGAA GAAAGGAA True
AATAGTTG AATAGTTG True
TAGCCTCC TAGCCTCC True
ACTACTGT ACTACTGT True
GGCCAGTT GG